In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cftime
import dask
import xarrayutils
import cartopy.crs as ccrs
from xmip.preprocessing import combined_preprocessing
from xmip.preprocessing import replace_x_y_nominal_lat_lon
from xmip.drift_removal import replace_time
from xmip.postprocessing import concat_experiments
import xmip.drift_removal as xm_dr
import xmip as xm
import xesmf as xe
import datetime
from dateutil.relativedelta import relativedelta
import utils
import cf_xarray as cfxr

from sklearn.linear_model import LinearRegression
import scipy.signal as signal
from scipy import stats
from datetime import timedelta

import seaborn as sns
import matplotlib as mpl
import cmocean
import cmocean.cm as cmo
from matplotlib.gridspec import GridSpec

from matplotlib.lines import Line2D
import matplotlib.patches as mpatches

import string

In [2]:
dask.config.set(**{'array.slicing.split_large_chunks': True})

## Import G

In [3]:
G_ds = xr.open_dataset('Outputs/G_ds.nc4')['__xarray_dataarray_variable__']
G_mean_ds = xr.open_dataset('Outputs/G_mean_ds.nc4')['__xarray_dataarray_variable__']

G_CDR_ds = xr.open_dataset('Outputs/G_cdr_ds.nc4')['__xarray_dataarray_variable__']
G_CDR_mean_ds = xr.open_dataset('Outputs/G_cdr_mean_ds.nc4')['__xarray_dataarray_variable__']

In [4]:
G_ds = xr.concat([G_ds, -G_CDR_ds], pd.Index(['pulse','cdr'], name = 'pulse_type'))
G_mean_ds = xr.concat([G_mean_ds, -G_CDR_mean_ds], pd.Index(['pulse','cdr'], name = 'pulse_type'))

In [5]:
G_ds.name = 'G[tas]'
G_mean_ds.name = 'G[tas]'

In [6]:
G_ds = G_ds.rename({'year':'s'})
G_mean_ds = G_mean_ds.rename({'year':'s'})

## Notes:

GFDL: 1pct and esm pi-control start from year 0001

UKESM1: 1pct starts in 1850 and pi-control starts in 1960, move 1pct to start in 1960

MIROC: both start from 1850

NORESM2: 1pct from 0001 pi-control from 1600-- move 1pct to 1600

ACCESS: 1pct and pi-control from 0101

CANESM5_r1p2: 1pct 1850, pi-control 5550, move 1pct to 5550


In [7]:
model_run_SSP37_dict = {'GFDL': 'GFDL-ESM4_ssp370_r1i1p1f1**'}
model_run_historical_dict = {'GFDL': 'GFDL-ESM4_historical_r1i1p1f1**'}
model_run_control_dict = {'GFDL': 'GFDL-ESM4_piControl_r1i1p1f1**'}


In [8]:
model_color = utils.model_color
type_color = utils.type_color

In [9]:
#define our output grid size

ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-89.5, 90.5, 1.0)),
        "lon": (["lon"], np.arange(0, 360, 1)),
        "lat_b": (["lat_b"], np.arange(-90.,91.,1.0)),
        "lon_b":(["lon_b"], np.arange(.5, 361.5, 1.0))
    }
)

In [10]:
A = utils.find_area(ds_out)

In [14]:
tas_SSP37 = {}
tas_pictrl = {}

for m in model_run_SSP37_dict.keys():
    print(m)
    print('tas')
    tas_past = xr.open_mfdataset(f'cmip6_data/historical/tas_Amon_{model_run_historical_dict[m]}',  use_cftime=True) #kg/m2/s
    tas_future = xr.open_mfdataset(f'cmip6_data/SSP37/tas_Amon_{model_run_SSP37_dict[m]}',  use_cftime=True) #kg/m2/s
    tas_SSP37[m] = xr.concat([tas_past,tas_future],dim = 'time')
    
    lat_corners = cfxr.bounds_to_vertices(tas_SSP37[m].isel(time = 0)['lat_bnds'], "bnds", order=None)
    lon_corners = cfxr.bounds_to_vertices(tas_SSP37[m].isel(time = 0)['lon_bnds'], "bnds", order=None)
    tas_SSP37[m] = tas_SSP37[m].assign(lon_b=lon_corners, lat_b=lat_corners)
    tas_SSP37[m] = utils._regrid_ds(tas_SSP37[m], ds_out)


GFDL
tas


/home/cwomack/.conda/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


In [17]:
for m in model_run_control_dict.keys():
    print(m)
    print('tas')
    tas_pictrl[m] = xr.open_mfdataset(f'cmip6_data/piControl/tas_Amon_{model_run_control_dict[m]}',  use_cftime=True) #kg/m2/s
    lat_corners = cfxr.bounds_to_vertices(tas_pictrl[m].isel(time = 0)['lat_bnds'], "bnds", order=None)
    lon_corners = cfxr.bounds_to_vertices(tas_pictrl[m].isel(time = 0)['lon_bnds'], "bnds", order=None)
    tas_pictrl[m] = tas_pictrl[m].assign(lon_b=lon_corners, lat_b=lat_corners)
    tas_pictrl[m] = utils._regrid_ds(tas_pictrl[m], ds_out)


GFDL
tas


/home/cwomack/.conda/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


In [24]:
# Align time between model runs and pi control runs
m = 'GFDL'
tas_SSP37[m]['time'] = tas_SSP37[m]['time'] - timedelta(365*1849)
tas_pictrl[m] = tas_pictrl[m].loc[dict(time = slice(tas_pictrl[m]['time'].min(), tas_SSP37[m]['time'].max()))]

In [26]:
dif_SSP37 = {}
for m1 in model_run_SSP37_dict.keys():
    if m1 == 'UKESM1_r1' or m1 == 'UKESM1_r2' or m1 == 'UKESM1_r3' or m1 == 'UKESM1_r4':
        m2 = 'UKESM1_r1'
    elif m1 == 'CANESM5_r1p1' or m1 == 'CANESM5_r2p1' or m1 == 'CANESM5_r3p1':
         m2 = 'CANESM5_r1p1'
    elif m1 == 'CANESM5_r1p2' or m1 == 'CANESM5_r2p2' or m1 == 'CANESM5_r3p2':
         m2 = 'CANESM5_r1p2'
    else:
        m2 = m1
    print(m1, m2)
    
    dif_SSP37[m1] = tas_SSP37[m1] - tas_pictrl[m2]

    if len(dif_SSP37[m1]['time']) > 3000:  #hack to get the time stamping to work, should find better fix
        periods = 3000
    else:
        periods = len(dif_SSP37[m1]['time'])
        
    times = pd.date_range('2000', periods= periods, freq='MS')
    weights = times.shift(1, 'MS') - times
    weights = xr.DataArray(weights, [('time', dif_SSP37[m1]['time'][:periods].values)]).astype('float')
    dif_SSP37[m1] =  (dif_SSP37[m1] * weights).groupby('time.year').sum('time')/weights.groupby('time.year').sum('time')

    dif_SSP37[m1]['year'] = range(len(dif_SSP37[m1]['year']))

GFDL GFDL


In [27]:
for m in dif_SSP37.keys():
    dif_SSP37[m] = dif_SSP37[m].drop('height')

In [28]:
#get rid of height and limit the time to the length of the GF
for m1 in ['UKESM1_r1', 'UKESM1_r2', 'UKESM1_r3', 'UKESM1_r4', 'NORESM2',
       'GFDL', 'MIROC', 'CANESM5_r1p2', 'CANESM5_r2p2', 'ACCESS', 'CANESM5_r3p2']:
    
    for t in ['pulse','cdr']:
        if m1 == 'UKESM1_r1' or m1 == 'UKESM1_r2' or m1 == 'UKESM1_r3' or m1 == 'UKESM1_r4':
            m2 = 'UKESM1_r1'
        elif m1 == 'CANESM5_r1p1' or m1 == 'CANESM5_r2p1' or m1 == 'CANESM5_r3p1':
             m2 = 'CANESM5_r1p1'
        elif m1 == 'CANESM5_r1p2' or m1 == 'CANESM5_r2p2' or m1 == 'CANESM5_r3p2':
             m2 = 'CANESM5_r1p2'
        else:
            m2 = m1
        
        length = len(G_ds.sel(model = m2, pulse_type = t).dropna(dim = 's')['s'])
        dif_SSP37[m] = dif_SSP37[m].sel(year = slice(0,length))


In [44]:
ds_dif_SSP37 = xr.concat([dif_SSP37[m] for m in dif_SSP37.keys()], pd.Index([m for m in dif_SSP37.keys()], name='model'), coords='minimal')


In [45]:
ds_dif = xr.concat([ds_dif_SSP37], pd.Index(['SSP37'], name='experiment'), coords='minimal')
ds_dif = ds_dif.rename({'year':'s'})


## Emissions profile

In [31]:
emis_profile_SSP37 = xr.open_dataset(f'SSP37_emis_profile_full.nc4')
emis_profile_SSP37 = emis_profile_SSP37.rename({'__xarray_dataarray_variable__':'emis'})

In [46]:
emis_profile = xr.concat([emis_profile_SSP37], pd.Index(['SSP37'], name='experiment'), coords='minimal')


## Define our Model Weights

In [47]:
#define our weights for models (grouping UKESM and CANESM realizations)
model_weights = {'GFDL': 1}
model_weights = xr.DataArray(
    data=list(model_weights.values()),
    dims=["model"],
    coords=dict(
        model=(["model"], list(model_weights.keys()))
    ),
    attrs=dict(
        description="weights for models"
    ),
)

In [48]:
G_model_weights = {'GFDL': 1}
G_model_weights = xr.DataArray(
    data=list(G_model_weights.values()),
    dims=["model"],
    coords=dict(
        model=(["model"], list(G_model_weights.keys()))
    ),
    attrs=dict(
        description="weights for Green's function"
    ),
)

## PiCtrl

In [49]:

#combine our picontrol data into one dataset, normalizing the time to year 0
pictrl = {}
for m in tas_pictrl.keys():    
    times = tas_pictrl[m].time.get_index('time')
    weights = times.shift(-1, 'MS') - times.shift(1, 'MS')
    weights = xr.DataArray(weights, [('time', tas_pictrl[m]['time'].values)]).astype('float')
    pictrl[m] =  (tas_pictrl[m] * weights).groupby('time.year').sum('time')/weights.groupby('time.year').sum('time')
    pictrl[m]['year'] = pictrl[m]['year'] - pictrl[m]['year'][0] 
    
for m in pictrl.keys():
    pictrl[m] = pictrl[m].drop('height')
ds_pictrl = xr.concat([pictrl[m] for m in pictrl.keys()], pd.Index([m for m in pictrl.keys()], name='model'), coords='minimal')


## Global Mean Analysis

In [50]:
%%time
GF = G_ds.weighted(A).mean(dim = ['lat','lon'])

conv_mean = {}
for exp in ['SSP37']:
    conv_mean[exp] = {}
    for m1 in ['GFDL']:
        conv_mean[exp][m1] = {}
        for t in ['pulse','cdr']:
            if m1 == 'UKESM1_r1' or m1 == 'UKESM1_r2' or m1 == 'UKESM1_r3' or m1 == 'UKESM1_r4':
                m2 = 'UKESM1_r1'
            else:
                m2 = m1
            conv_mean[exp][m1][t] = signal.convolve( np.array(GF.sel(model = m2, pulse_type = t).dropna(dim = 's')), np.array(emis_profile.sel(model = m1, experiment = exp)['emis']),'full')
            conv_mean[exp][m1][t] = utils.np_to_xr_mean(conv_mean[exp][m1][t], GF.sel(model = m2, pulse_type = t), emis_profile.sel(model = m1, experiment = exp))
            length = len(G_ds.weighted(A).mean(dim = ['lat','lon']).dropna(dim = 's')['s'])
            conv_mean[exp][m1][t] = conv_mean[exp][m1][t][:length]


CPU times: user 4.06 s, sys: 805 ms, total: 4.87 s
Wall time: 4.88 s


In [41]:
#convert to dataset

conv_dict = {}
for exp in conv_mean.keys():
    conv_dict[exp] = {}
    for m in conv_mean[exp].keys():
        conv_dict[exp][m] = xr.concat([conv_mean[exp][m][t] for t in conv_mean[exp][m].keys()], pd.Index([t for t in conv_mean[exp][m].keys()], name='pulse_type'), coords='minimal')
for exp in conv_mean.keys():
    conv_dict[exp] = xr.concat([conv_dict[exp][m] for m in conv_mean[exp].keys()], pd.Index([m for m in conv_mean[exp].keys()], name='model'), coords='minimal')
conv_mean_ds = xr.concat([conv_dict[exp] for exp in conv_dict.keys()], pd.Index([exp for exp in conv_dict.keys()], name='experiment'), coords='minimal')


In [53]:

def reindex_df(df, weight_col):
    """expand the dataframe to prepare for resampling
    result is 1 row per count per sample"""
    df = df.reindex(df.index.repeat(df[weight_col]))
    df.reset_index(drop=True, inplace=True)
    return(df)

data = {'mean': conv_mean_ds.sel(experiment = 'SSP37').mean(dim = ['pulse_type']).sel(s = slice(60,80)).mean(dim = ['s']).values,
        'model': conv_mean_ds.model.values,
        'count': [12]}
df_SSP37_emulator = pd.DataFrame(data)
df_SSP37_emulator = reindex_df(df_SSP37_emulator, weight_col = 'count')

data = {'mean': ds_dif.where(ds_dif['model'].isin(model_weights.model.values), drop = True).sel(experiment = 'SSP37').sel(s = slice(60,80)).weighted(A).mean(dim = ['s', 'lon','lat'])['tas'].values,
        'model': ds_dif.where(ds_dif['model'].isin(model_weights.model.values), drop = True).sel(experiment = 'SSP37').model.values,
        'count': [12]}

df_SSP37_model = pd.DataFrame(data)
df_SSP37_model = reindex_df(df_SSP37_model, weight_col = 'count')

#######ZEC############

data = {'mean': (conv_mean_ds.sel(experiment = 'SSP37').where(conv_mean_ds['model'].isin(list(model_run_SSP37_dict.keys())), drop = True).mean(dim = 'pulse_type').sel(
                                                                s = slice(80-10, 80+10)).mean(dim = 's').weighted(A).mean(dim = ['lat','lon']) - 
                    conv_mean_ds.sel(experiment = 'SSP37').where(conv_mean_ds['model'].isin(list(model_run_SSP37_dict.keys())), drop = True).mean(dim = 'pulse_type').sel(s = 65).weighted(A).mean(dim = ['lat','lon'])).values,
        'model': conv_mean_ds.where(conv_mean_ds['model'].isin(list(model_run_SSP37_dict.keys())), drop = True).model.values,
        'count': [12]}

df_zec_emulator_model = pd.DataFrame(data)
df_zec_emulator_model = reindex_df(df_zec_emulator_model, weight_col = 'count')

data = {'mean': (ds_dif.where(ds_dif['model'].isin(list(model_run_SSP37_dict.keys())), drop = True).sel(experiment = 'SSP37').sel(
                                                                s = slice(80-10, 80+10)).mean(dim = 's').weighted(A).mean(dim = ['lat','lon']) - 
                    ds_dif.where(ds_dif['model'].isin(list(model_run_SSP37_dict.keys())), drop = True).sel(experiment = 'SSP37').sel(s = 65).weighted(A).mean(dim = ['lat','lon']))['tas'].values,
        
        'model': ds_dif.where(ds_dif['model'].isin(list(model_run_SSP37_dict.keys())), drop = True).sel(experiment = 'SSP37').model.values,
        'count': [12]}

df_zec_cmip_model = pd.DataFrame(data)
df_zec_cmip_model = reindex_df(df_zec_cmip_model, weight_col = 'count')

## Spatial Analysis

In [32]:
%%time

GF = G_ds

conv = {}
for exp in ['SSP37']:
    print(exp)
    conv[exp] = {}
    if exp == 'SSP37':
        model_list = [CANESM5_r2p2', 'ACCESS', 'CANESM5_r3p2',]
    elif exp == '1pct':
        model_list = ['UKESM1_r1', 'UKESM1_r2', 'UKESM1_r3', 'UKESM1_r4', 'NORESM2', 'GFDL', 'MIROC', 'CANESM5_r1p2', 'CANESM5_r2p2', 'ACCESS', 'CANESM5_r3p2',]
    for m1 in model_list:
        conv[exp][m1] = {}
        for t in ['pulse','cdr']:
            if m1 == 'UKESM1_r1' or m1 == 'UKESM1_r2' or m1 == 'UKESM1_r3' or m1 == 'UKESM1_r4':
                m2 = 'UKESM1_r1'
            else:
                m2 = m1
            print(m1, m2)
            conv[exp][m1][t] = signal.convolve(np.array(GF.sel(model = m2, pulse_type = t).dropna(dim = 's')), 
                                               np.array(emis_profile.sel(model = m1, experiment = exp)['emis'])[~np.isnan(np.array(emis_profile.sel(model = m1, experiment = exp)['emis']))][..., None, None],
                                               'full')
            conv[exp][m1][t] = utils.np_to_xr(conv[exp][m1][t], GF.sel(model = m2, pulse_type = t), emis_profile.sel(model = m1, experiment = exp))


1000gtc
UKESM1_r1 UKESM1_r1
UKESM1_r1 UKESM1_r1
UKESM1_r2 UKESM1_r1
UKESM1_r2 UKESM1_r1
UKESM1_r3 UKESM1_r1
UKESM1_r3 UKESM1_r1
UKESM1_r4 UKESM1_r1
UKESM1_r4 UKESM1_r1
NORESM2 NORESM2
NORESM2 NORESM2
MIROC MIROC
MIROC MIROC
CANESM5_r1p2 CANESM5_r1p2
CANESM5_r1p2 CANESM5_r1p2
CANESM5_r2p2 CANESM5_r2p2
CANESM5_r2p2 CANESM5_r2p2
ACCESS ACCESS
ACCESS ACCESS
CANESM5_r3p2 CANESM5_r3p2
CANESM5_r3p2 CANESM5_r3p2
1pct
UKESM1_r1 UKESM1_r1
UKESM1_r1 UKESM1_r1
UKESM1_r2 UKESM1_r1
UKESM1_r2 UKESM1_r1
UKESM1_r3 UKESM1_r1
UKESM1_r3 UKESM1_r1
UKESM1_r4 UKESM1_r1
UKESM1_r4 UKESM1_r1
NORESM2 NORESM2
NORESM2 NORESM2
GFDL GFDL
GFDL GFDL
MIROC MIROC
MIROC MIROC
CANESM5_r1p2 CANESM5_r1p2
CANESM5_r1p2 CANESM5_r1p2
CANESM5_r2p2 CANESM5_r2p2
CANESM5_r2p2 CANESM5_r2p2
ACCESS ACCESS
ACCESS ACCESS
CANESM5_r3p2 CANESM5_r3p2
CANESM5_r3p2 CANESM5_r3p2
CPU times: user 21.6 s, sys: 24.5 s, total: 46.1 s
Wall time: 46.2 s


In [33]:
#convert to dataset

conv_dict = {}
for exp in conv.keys():
    conv_dict[exp] = {}
    for m in conv[exp].keys():
        conv_dict[exp][m] = xr.concat([conv[exp][m][t] for t in conv[exp][m].keys()], pd.Index([t for t in conv[exp][m].keys()], name='pulse_type'), coords='minimal')
for exp in conv.keys():
    conv_dict[exp] = xr.concat([conv_dict[exp][m] for m in conv[exp].keys()], pd.Index([m for m in conv[exp].keys()], name='model'), coords='minimal')
conv_ds = xr.concat([conv_dict[exp] for exp in conv_dict.keys()], pd.Index([exp for exp in conv_dict.keys()], name='experiment'), coords='minimal')


# Save out

In [34]:
conv_mean_ds.to_netcdf('Outputs/conv_mean_ds.nc4')

conv_ds.to_netcdf('Outputs/conv_ds.nc4')

emis_profile.to_netcdf('Outputs/emis_profile.nc4')

ds_dif.to_netcdf('Outputs/ds_dif.nc4')

In [36]:
ds_pictrl.to_netcdf('Outputs/ds_pictrl.nc4')